In [1]:
import numpy as np
from os import urandom
from tensorflow.keras.regularizers import l2
from tensorflow.keras.backend import concatenate
from tensorflow.keras import backend as K
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, AveragePooling1D, Conv1D, MaxPooling1D, Input, Reshape, Permute, Add, Flatten, BatchNormalization, Activation, LeakyReLU
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from pickle import dump
import tensorflow as tf
import tensorflow
from sklearn.model_selection import train_test_split

def cyclic_lr(num_epochs, high_lr, low_lr):
    res = lambda i: low_lr + ((num_epochs-1) - i % num_epochs)/(num_epochs-1) * (high_lr - low_lr)
    return res

def make_checkpoint(datei):
    res = ModelCheckpoint(datei, monitor='val_loss', save_best_only = True)
    return res

bs = 1024

In [2]:
### make the linear dataset: run the des_linear_dataset.cpp

In [3]:
def make_dataset_from_txt(path):
    X = []
    Y = []
    with open(path, 'r') as file:
        for line in file:
            values = line.split()
            binary_values = [bin(int(val, 16))[2:].zfill(64) for val in values[:4]]
            x_row = ''.join(binary_values)
            X.append(np.array([int(bit) for bit in x_row]))
            Y.append(int(values[4]))
    return np.array(X), np.array(Y)

In [4]:
### define the resnet model

In [5]:
def make_resnet(num_blocks=2, num_filters=32, num_outputs=1, ds=[64, 64], word_size=64, ks=3, depth=1, reg_param=0.00001, final_activation='sigmoid'):
    inp = Input(shape=(num_blocks * word_size * 2,))
    rs = Reshape((2 * num_blocks, word_size))(inp)
    perm = Permute((2,1))(rs)
    conv0 = Conv1D(num_filters, kernel_size=1, padding='same', kernel_regularizer=l2(reg_param))(perm)
    conv0 = BatchNormalization()(conv0)
    conv0 = Activation('relu')(conv0)
    shortcut = conv0
    for i in range(depth):
        conv1 = Conv1D(num_filters, kernel_size=ks, padding='same', kernel_regularizer=l2(reg_param))(shortcut)
        conv1 = BatchNormalization()(conv1)
        conv1 = Activation('relu')(conv1)
        conv2 = Conv1D(num_filters, kernel_size=ks, padding='same',kernel_regularizer=l2(reg_param))(conv1)
        conv2 = BatchNormalization()(conv2)
        conv2 = Activation('relu')(conv2)
        shortcut = Add()([shortcut, conv2])
    flat = Flatten()(shortcut)
    for d in ds:
        dense = Dense(d,kernel_regularizer=l2(reg_param))(flat)
        dense = BatchNormalization()(dense)
        dense = Activation('relu')(dense)
    out = Dense(num_outputs, activation=final_activation, kernel_regularizer=l2(reg_param))(dense)
    model = Model(inputs=inp, outputs=out)
    return model

In [6]:
### train the resnet

In [7]:
net = make_resnet()
net.compile(optimizer='adam', loss='mse', metrics=['acc'])
_X, _Y = make_dataset_from_txt('des_linear_dataset_0.txt')
X, X_eval, Y, Y_eval = train_test_split(_X, _Y, test_size=0.2, random_state=42)
check = make_checkpoint('./freshly_trained_nets/best_linear_net.h5')
lr = LearningRateScheduler(cyclic_lr(10,0.002, 0.0001))
h = net.fit(X, Y, epochs=20, batch_size=bs, validation_data=(X_eval, Y_eval), callbacks=[lr,check])
print("Best validation accuracy: ", np.max(h.history['val_acc']))

Epoch 1/20
782/782 [==============================] - 29s 27ms/step - loss: 0.2509 - acc: 0.5443 - val_loss: 0.2444 - val_acc: 0.5639
Epoch 2/20
  5/782 [..............................] - ETA: 20s - loss: 0.2344 - acc: 0.6475

C:\UserSoftwareSpace\Anaconda3\envs\tensorflow26\lib\site-packages\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


782/782 [==============================] - 20s 26ms/step - loss: 0.2230 - acc: 0.6816 - val_loss: 0.2184 - val_acc: 0.6881
Epoch 3/20
782/782 [==============================] - 20s 26ms/step - loss: 0.2166 - acc: 0.6911 - val_loss: 0.2146 - val_acc: 0.6945
Epoch 4/20
782/782 [==============================] - 20s 26ms/step - loss: 0.2143 - acc: 0.6934 - val_loss: 0.2157 - val_acc: 0.6892
Epoch 5/20
782/782 [==============================] - 20s 26ms/step - loss: 0.2134 - acc: 0.6939 - val_loss: 0.2127 - val_acc: 0.6965
Epoch 6/20
782/782 [==============================] - 20s 26ms/step - loss: 0.2126 - acc: 0.6943 - val_loss: 0.2121 - val_acc: 0.6951
Epoch 7/20
782/782 [==============================] - 20s 26ms/step - loss: 0.2120 - acc: 0.6947 - val_loss: 0.2134 - val_acc: 0.6953
Epoch 8/20
782/782 [==============================] - 21s 26ms/step - loss: 0.2114 - acc: 0.6949 - val_loss: 0.2106 - val_acc: 0.6974
Epoch 9/20
782/782 [==============================] - 20s 26ms/step - los

In [8]:
### test the resnet

In [9]:
net.load_weights('./freshly_trained_nets/best_linear_net.h5')
X_test, Y_test = make_dataset_from_txt('des_linear_dataset_1.txt')
net.evaluate(X_test, Y_test, batch_size=bs, verbose=1)

977/977 [==============================] - 7s 7ms/step - loss: 0.2101 - acc: 0.6955


[0.2100670039653778, 0.6954889893531799]